# Activity and Code

In [2]:
import pandas as pd

# Replace 'your_file_path.csv' with the path to the CSV file you want to read
# file_path = '/Users/ruchiraray/Documents/UT Austin/Soc-HHR/BEHAVIOR1kData/survey_analysis/lexiconwex2022.csv'
file_path = "/Users/ruchiraray/Documents/UT Austin/Soc-HHR/lexiconwex2021.csv"

# Attempt to read the CSV file

df = pd.read_csv(file_path, header=0)
activity_code_df = df[df['6-digit activity code'].notna()][['6-digit activity code', 'Activity']]

activity_code_dict = pd.Series(activity_code_df['Activity'].values, index=activity_code_df['6-digit activity code']).to_dict()
print(len(activity_code_dict))

466


# ATUS TIME

In [4]:
import pandas as pd
atus_summary_df = pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/atussum-2021/atussum_2021.csv', header=0)

# ATUS WB

### Filter

In [5]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/atusrost-2021/atusrost_2021.csv')

# Group by 'TUCASEID' and count the unique 'TULINENO' entries for each 'TUCASEID'
household_counts = df.groupby('TUCASEID')['TULINENO'].nunique().reset_index()

# Rename columns to reflect the data
household_counts.columns = ['TUCASEID', 'Number of Household Members']

# Save the result to a new CSV file
# household_counts.to_csv('household_members_ATUS.csv', index=False)

## Filter by Sex, Income Range, Age Range, Race, Number of Household Children, 

In [7]:

sex_dict = {
    'male': 1,
    'female': 2
}

sex="female"


income_mapping = {
    1:[0,5000],
    2:[5000,7499],
    3:[7500,9999],
    4:[10000,12499],
    5:[12500,14999],
    6:[15000,19999],
    7:[20000,24999],
    8:[25000,29999],
    9:[30000,34999],
    10:[35000,39999],
    11:[40000,49999],
    12:[50000,59999],
    13:[60000,74999],
    14:[75000,99999],
    15:[100000,149999],
    16:[150000,9999999999]
}

income= None #"1"

income_class=None #"high"

start_age = 0
end_age = 1000


race_dict = {
    'Asian Indian': 1,
    'Chinese': 2,
    'Filipino': 3,
    'Japanese': 4,
    'Korean': 5,
    'Vietnamese': 6,
    'Other': 7
}
race_name = None #'Chinese'


# Sex


if sex:
    sex_number = sex_dict[sex]
    atus_summary_df=atus_summary_df[atus_summary_df['TESEX'] == sex_number]



# Income Claasification by family member threshold + poverty line 2021
if income_class:   
    # Load the data
    atuscps_df = pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/atuscps-2021/atuscps_2021.csv')
    household_members_df = household_counts
    # pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/BEHAVIOR1kData/survey_analysis/household_members_ATUS.csv')
    thresh_df = pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/BEHAVIOR1kData/survey_analysis/thresholds.csv')

    # Merge atuscps_df with household_members_df to get the number of members for each household
    atuscps_df = atuscps_df.merge(household_members_df, on='TUCASEID')

    # Function to classify income based on HEFAMINC, number of members, and thresholds
    def classify_income(row, thresh_df):
        income_code = row['HEFAMINC']
        members = row['Members']
        
        # Map the income code to a range using income_mapping
        lower_limit, upper_limit = income_mapping[income_code]

        # Find the appropriate threshold for the number of members
        if members >= 9:  # Assuming last row in thresh_df is for 5 and above
            thresh_row = thresh_df.iloc[-1]
        else:
            thresh_row = thresh_df[thresh_df['Size'] == members].iloc[0]

        # # Classify income
        # if lower_limit==0:
        #     return 'low'
        # else:
        # Determine if it's mid based on the mapping and thresholds
        low_thresh = thresh_row['poverty']
        high_thresh = thresh_row['high']
        if upper_limit <= low_thresh:
            return 'low'
        elif lower_limit >= high_thresh:
            return 'high'
        else:
            return 'mid'

    # Apply the classification function
    atuscps_df['Income Classification'] = atuscps_df.apply(classify_income, args=(thresh_df,), axis=1)

    atus_summary_df = atus_summary_df.merge(atuscps_df[['TUCASEID', 'Income Classification']], on='TUCASEID', how='left')

    atus_summary_df=atus_summary_df[atus_summary_df['Income Classification'] == "high"]

# Income

if income:
    # Load the data
    atuscps_df = pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/atuscps-2021/atuscps_2021.csv')

    # Merge atussum_df with classified_atuscps_df to get HEFAMINC value
    atus_summary_df = atus_summary_df.merge(atuscps_df[['TUCASEID', 'HEFAMINC']], on='TUCASEID', how='left')

    # Filter rows where HEFAMINC is 2
    atus_summary_df = atus_summary_df[atus_summary_df['HEFAMINC'] == income]



# Income Range -custom 

# Age Range
start_age = 0
# int(input("Enter the start age: "))
end_age = 1000
# int(input("Enter the end age: "))
atus_summary_df = atus_summary_df[(atus_summary_df['TEAGE'] >= start_age) & (atus_summary_df['TEAGE'] <= end_age)]


# Race


if race_name:
    race_number = race_dict[race_name]
    atus_summary_df=atus_summary_df[atus_summary_df['PTDTRACE'] == race_number]



# Number of Household Children - no direct way



In [8]:
import pandas as pd

# Load the datasets
atusact_df = pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/atusact-2021/atusact_2021.csv',header=0)
# atussum_df = pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/atussum-2021/atussum_2021.csv')


# Merge the filtered atussum_df with atusact_df on TUCASEID
atusact_subset_df = pd.merge(atusact_df, atus_summary_df[['TUCASEID']], on='TUCASEID', how='inner')

wb = pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/wbact-2021/wbact_2021.csv')

# Merge the datasets on TUCASEID and TUACTIVITY_N
merged_data = pd.merge(atusact_subset_df, wb, on=['TUCASEID', 'TUACTIVITY_N'])
merged_data

,TUCASEID,TUACTIVITY_N,TEWHERE,TRTCCTOT_LN,TRTCC_LN,TRTCOC_LN,TRTEC_LN,TRTHH_LN,TRTNOHH_LN,TRTOHH_LN,...,WUINTERACT,WUMEANING,WUPAIN,WUPNORD,WUSAD,WUSADORD,WUSTRESS,WUSTRORD,WUTIRED,WUTRDORD
0,20210301210166,3,3,0,-1,0,0,-1,-1,-1,...,2,4,2,3,3,5,3,1,4,2
1,20210301210166,9,1,0,-1,0,0,-1,-1,-1,...,1,4,0,3,0,5,0,1,2,2
2,20210301210166,11,7,0,-1,0,0,-1,-1,-1,...,1,4,0,3,0,5,2,1,0,2
3,20210301210218,2,1,0,-1,0,-1,-1,-1,-1,...,1,6,0,1,0,3,4,4,6,5
4,20210301210218,5,1,0,-1,0,-1,-1,-1,-1,...,2,0,0,1,0,3,0,4,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11086,20211212212502,5,1,0,-1,0,-1,-1,-1,-1,...,1,5,0,5,0,3,0,4,1,2
11087,20211212212502,10,1,0,-1,0,-1,-1,-1,-1,...,1,3,0,5,0,3,0,4,1,2
11088,20211212212567,3,1,240,240,0,-1,240,-1,240,...,1,3,3,5,5,4,6,3,5,1
11089,20211212212567,11,1,20,20,0,-1,20,-1,20,...,1,6,2,5,3,4,2,3,5,1


In [50]:


filtered_data = merged_data[(merged_data['WUHAPPY'] >= 0) & (merged_data['WUMEANING'] >= 0)]

# Group by 'TRCODE' and calculate the mean for 'WUHAPPY' and 'WUMEANING'
mean_values = filtered_data.groupby('TRCODE')[['WUHAPPY', 'WUMEANING']].mean().reset_index()


activity_time = pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/BEHAVIOR1kData/survey_analysis/activity_time_2021_mid.csv')

# Ensure the TRCODE and Activity Code are of the same type, adjust as necessary
activity_time['Activity Code'] = activity_time['Activity Code'].astype(int)

# Merge the DataFrames on the appropriate columns
merged_data_with_activities = pd.merge(mean_values, activity_time, left_on='TRCODE', right_on='Activity Code')

# Select and rename the columns as needed
final_data = merged_data_with_activities[['TRCODE', 'WUHAPPY', 'WUMEANING', 'Activity']]

